In [1]:
import pennylane as qml
from pennylane import numpy as np

## A.3.1

Concept: Consider $U_f\ket{\vec{x}+}$, where $\vec{x}+$ has the same length as $\vec{s}$. Paying attention to how $U_f$ transforms the state, we obtain $\ket{\vec{x}-}$ exactly when $\vec{x}$ is a prefix of $\vec{s}$; otherwise, we still have $\ket{\vec{x}+}$. If we should apply Hadamard again to the last wire, we read $\ket{1}$ when $\vec{x}$ is a prefix and $\ket{0}$ otherwise.


In [ ]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def pair_circuit(x_tilde, combo):
    """Test a pair labelled by x_tilde for the presence of a solution.
    
    Args:
        x_tilde (list[int]): An (n_bits - 1)-string labelling the pair to test.
        combo (list[int]): A secret combination of n_bits 0s and 1s.
        
    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    ##################
    # YOUR CODE HERE #
    ##################
    qml.Hadamard(wires=n_bits-1)
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.Hadamard(wires=n_bits-1)
    
    return qml.probs(wires=n_bits-1)


## A.3.2

Concept: The significance of this is the preceding is that instead of making brute-force queries of $U_f$ against the $2^{|\vec{s}|}$ permutations of an $|\vec{s}|$-bit state (requiring $2^{|\vec{s}|}-1$ queries in the worst case and $(L-1)/2+(L-1)/L$ queries on average, where $L=2^{|\vec{s}|}$), we make queries against the $2^{|\vec{s}|-1}$ permutations of the $(|\vec{s}|-1)$-bit prefixes of $\vec{s}$, requiring $(L-1)/2-1/L$ tests on average, where $L=2^{|\vec{s}|-1}$.

Note: we test each permutation in order except for the last, which we know automatically should it be the secret from every other permutation failing; when testing in pairs, we need to run an additional test once we have identified the pair, to determine which of the two it is.

In [ ]:
def pair_lock_picker(trials):
    """Create a combo, run pair_circuit until it succeeds, and tally success rate.
    
    Args:
        trials (int): Number of times to test the lock picker.

    Returns:
        float: The average number of times the lock picker uses pair_circuit.
    """
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs] 

    test_numbers = []

    for trial in range(trials):
        combo = secret_combo(n_bits) # Random list of bits
        counter = 0
        for x_tilde in x_tildes:
            counter += 1

            ##################
            # YOUR CODE HERE #
            ##################
            if np.isclose(pair_circuit(x_tilde, combo), np.array([0.+0.j, 1.+0.j])).all():
                break
        
        test_numbers.append(counter)
    return sum(test_numbers)/trials

trials = 500
output = pair_lock_picker(trials)

print(f"For {n_bits} bits, it takes", output, "pair tests on average.")
